In [1]:
# Import necessary libraries
import dash
import dash_bootstrap_components as dbc
from dash import html, dcc
import pandas as pd
import plotly.graph_objects as go
from dash.dependencies import Input, Output

# Load the dataset
file_path = 'fifa_players.csv'  # Ensure this path is correct
fifa_df = pd.read_csv(file_path)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define available stats
stats_columns = [
    'crossing', 'finishing', 'heading_accuracy', 'short_passing',
    'volleys', 'dribbling', 'curve', 'freekick_accuracy',
    'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
    'agility', 'reactions', 'balance', 'shot_power',
    'jumping', 'stamina', 'strength', 'long_shots', 
    'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
    'composure', 'marking', 'standing_tackle', 'sliding_tackle'
]

# Define the layout of the app with navigation
app.layout = dbc.Container(
    [
        dcc.Location(id='url', refresh=False),
        dbc.NavbarSimple(
            brand="FIFA Player Stats Dashboard",
            brand_href="/",
            color="primary",
            dark=True,
            children=[
                dbc.NavLink("Player Stats", href="/player-stats", active="exact"),
                dbc.NavLink("Player Comparison", href="/player-comparison", active="exact"),
            ],
        ),
        html.Div(id='page-content'),
    ],
    fluid=True
)

# Page for displaying individual player stats
player_stats_layout = html.Div(
    [
        dbc.Row(
            dbc.Col(html.H2("Player Stats"), className="text-center"),
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                dcc.Dropdown(
                    id='player-dropdown',
                    options=[{'label': name, 'value': name} for name in fifa_df['name']],
                    value=fifa_df['name'].iloc[0],  # Default value
                    multi=False,
                    clearable=False,
                    placeholder="Select Player"
                ),
                width=12
            ),
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='bar-chart'),
                width=6
            ),
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='pie-chart'),
                width=6
            ),
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='line-chart'),
                width=12
            ),
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                html.Div(id='player-stats'),
                width=12
            ),
            className="mb-4"
        )
    ]
)

# Page for comparing two players
comparison_layout = html.Div(
    [
        dbc.Row(
            dbc.Col(html.H2("Player Comparison"), className="text-center"),
            className="mb-4"
        ),
        dbc.Row(
            [
                dbc.Col(
                    dcc.Dropdown(
                        id='player1-dropdown',
                        options=[{'label': name, 'value': name} for name in fifa_df['name']],
                        value=fifa_df['name'].iloc[0],  # Default value
                        multi=False,
                        clearable=False,
                        placeholder="Select Player 1"
                    ),
                    width=6
                ),
                dbc.Col(
                    dcc.Dropdown(
                        id='player2-dropdown',
                        options=[{'label': name, 'value': name} for name in fifa_df['name']],
                        value=fifa_df['name'].iloc[1],  # Default value
                        multi=False,
                        clearable=False,
                        placeholder="Select Player 2"
                    ),
                    width=6
                )
            ],
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='radar-chart'),
                width=12
            ),
            className="mb-4"
        ),
        dbc.Row(
            [
                dbc.Col(
                    dcc.Graph(id='comparison-bar-chart'),
                    width=6
                ),
                dbc.Col(
                    dcc.Graph(id='comparison-line-chart'),
                    width=6
                )
            ],
            className="mb-4"
        ),
        dbc.Row(
            dbc.Col(
                html.Div(id='comparison-stats'),
                width=12
            ),
            className="mb-4"
        )
    ]
)

# Callback to render the appropriate page based on URL
@app.callback(Output('page-content', 'children'), Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/player-stats':
        return player_stats_layout
    elif pathname == '/player-comparison':
        return comparison_layout
    else:
        return player_stats_layout  # Default to player stats page

# Callback to update the player stats page charts
@app.callback(
    Output('bar-chart', 'figure'),
    Output('pie-chart', 'figure'),
    Output('line-chart', 'figure'),
    Output('player-stats', 'children'),
    Input('player-dropdown', 'value')
)
def update_player_charts(player):
    # Get the selected player's stats
    player_stats = fifa_df[fifa_df['name'] == player].iloc[0]

    # Create the bar chart
    bar_fig = go.Figure()
    bar_fig.add_trace(go.Bar(
        x=stats_columns,
        y=[player_stats[stat] for stat in stats_columns],
        marker_color='blue'
    ))
    bar_fig.update_layout(title=f'{player} - Bar Chart of Stats', yaxis_title='Skill Value', xaxis_title='Stats')

    # Create the pie chart
    pie_fig = go.Figure()
    pie_fig.add_trace(go.Pie(
        labels=stats_columns,
        values=[player_stats[stat] for stat in stats_columns],
        hole=0.3
    ))
    pie_fig.update_layout(title=f'{player} - Pie Chart of Stats')

    # Create a line chart
    line_fig = go.Figure()
    line_fig.add_trace(go.Scatter(
        x=stats_columns,
        y=[player_stats[stat] for stat in stats_columns],
        mode='lines+markers',
        name=player,
        line=dict(color='blue')
    ))
    line_fig.update_layout(title=f'{player} - Line Chart of Stats', yaxis_title='Skill Value', xaxis_title='Stats')

    # Create detailed stats display
    stats_display = html.Div([
        html.H4(f"{player} Stats"),
        html.Ul([html.Li(f"{category}: {player_stats[category]}") for category in stats_columns]),
    ])

    return bar_fig, pie_fig, line_fig, stats_display

# Callback to update the radar and comparison charts based on the selected players
@app.callback(
    Output('radar-chart', 'figure'),
    Output('comparison-bar-chart', 'figure'),
    Output('comparison-line-chart', 'figure'),
    Output('comparison-stats', 'children'),
    Input('player1-dropdown', 'value'),
    Input('player2-dropdown', 'value')
)
def update_comparison_charts(player1, player2):
    # Get the selected players' stats
    player1_stats = fifa_df[fifa_df['name'] == player1].iloc[0]
    player2_stats = fifa_df[fifa_df['name'] == player2].iloc[0]

    # Define the categories for the radar chart
    categories = stats_columns

    # Values for the radar chart
    player1_values = [player1_stats[category] for category in categories]
    player2_values = [player2_stats[category] for category in categories]

    # Create the radar chart
    radar_fig = go.Figure()

    # Add the players' stats to radar chart
    radar_fig.add_trace(go.Scatterpolar(
        r=player1_values + [player1_values[0]],  # Close the circle
        theta=categories + [categories[0]],
        fill='toself',
        name=player1,
        line=dict(color='blue'),
        fillcolor='rgba(0, 0, 255, 0.25)'
    ))

    radar_fig.add_trace(go.Scatterpolar(
        r=player2_values + [player2_values[0]],  # Close the circle
        theta=categories + [categories[0]],
        fill='toself',
        name=player2,
        line=dict(color='red'),
        fillcolor='rgba(255, 0, 0, 0.25)'
    ))

    # Update layout for the radar chart
    radar_fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]  # Adjust this based on your data range
            )
        ),
        showlegend=True,
        title=f'Comparison of {player1} and {player2}'
    )

    # Create Bar chart for comparison
    comparison_bar_fig = go.Figure()
    comparison_bar_fig.add_trace(go.Bar(
        x=categories,
        y=[player1_stats[stat] for stat in categories],
        name=player1,
        marker_color='blue'
    ))
    comparison_bar_fig.add_trace(go.Bar(
        x=categories,
        y=[player2_stats[stat] for stat in categories],
        name=player2,
        marker_color='red'
    ))
    comparison_bar_fig.update_layout(title='Bar Chart Comparison', barmode='group', yaxis_title='Skill Value', xaxis_title='Stats')

    # Create Line chart for comparison
    comparison_line_fig = go.Figure()
    comparison_line_fig.add_trace(go.Scatter(
        x=categories,
        y=[player1_stats[stat] for stat in categories],
        mode='lines+markers',
        name=player1,
        line=dict(color='blue')
    ))
    comparison_line_fig.add_trace(go.Scatter(
        x=categories,
        y=[player2_stats[stat] for stat in categories],
        mode='lines+markers',
        name=player2,
        line=dict(color='red')
    ))
    comparison_line_fig.update_layout(title='Line Chart Comparison', yaxis_title='Skill Value', xaxis_title='Stats')

    # Create detailed stats display
    comparison_stats_display = html.Div([
        html.H4(f"{player1} Stats"),
        html.Ul([html.Li(f"{category}: {player1_stats[category]}") for category in stats_columns]),
        html.H4(f"{player2} Stats"),
        html.Ul([html.Li(f"{category}: {player2_stats[category]}") for category in stats_columns]),
    ])

    return radar_fig, comparison_bar_fig, comparison_line_fig, comparison_stats_display

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8070)
